In [2]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
from arcgis.features import FeatureLayer
import re


# Make Feature Layer

In [40]:
sdfWFForecasts = pd.DataFrame.spatial.from_featureclass('data-forecasts/wf/WFv901_Segments_20240226_Draft.shp')

sdfCombinedForecasts = pd.concat([sdfWFForecasts]) # add others later
sdfCombinedForecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0006_141.0,0006,141.035,146.868,5.843301,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,1,0,0006_146.9,0006,146.868,149.902,3.039023,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,2,0,0006_149.9,0006,149.902,150.580,0.677983,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,3,0,0006_150.6,0006,150.580,152.555,1.973104,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,4,0,0006_152.6,0006,152.555,152.871,0.316275,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,4975,0,WFRC_8469,WFRC,0.000,0.000,0.567664,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{""paths"": [[[407213.3742000004, 4504030.9891],..."
4976,4976,0,WFRC_8470,WFRC,0.000,0.000,0.493148,35.0,1.0,WFRC,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{""paths"": [[[407723.2000000002, 4503422.753000..."
4977,4977,0,WFRC_8471,WFRC,0.000,0.000,0.405891,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{""paths"": [[[415734.59509999957, 4481608.6051]..."
4978,4978,0,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5,1.0245,0.0,"{""paths"": [[[424308.5, 4489341.199999999], [42..."


In [41]:
sdfCombinedForecasts = sdfCombinedForecasts[sdfCombinedForecasts['ROUTE']!='UTA']
sdfCombinedForecasts


,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0006_141.0,0006,141.035,146.868,5.843301,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,1,0,0006_146.9,0006,146.868,149.902,3.039023,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,2,0,0006_149.9,0006,149.902,150.580,0.677983,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,3,0,0006_150.6,0006,150.580,152.555,1.973104,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,4,0,0006_152.6,0006,152.555,152.871,0.316275,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,4975,0,WFRC_8469,WFRC,0.000,0.000,0.567664,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{""paths"": [[[407213.3742000004, 4504030.9891],..."
4976,4976,0,WFRC_8470,WFRC,0.000,0.000,0.493148,35.0,1.0,WFRC,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{""paths"": [[[407723.2000000002, 4503422.753000..."
4977,4977,0,WFRC_8471,WFRC,0.000,0.000,0.405891,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{""paths"": [[[415734.59509999957, 4481608.6051]..."
4978,4978,0,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5,1.0245,0.0,"{""paths"": [[[424308.5, 4489341.199999999], [42..."


In [42]:
sdfCombinedForecasts.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\Traffic-Volume-Historic-and-Forecast.shp'

# Create Route Json

In [5]:
# Route Json
# R: Route Label (string, first 4 of SEGID)
# C: CO_FIPS (integer)
# label: Prefix of 
#        "I-"  + Route Label with 0s removed : for interstates
#        "Hwy" + Route Label with 0s removed : us and state routes
#        "FA"  + Route Label with 0s removed : federal aid routes
#        "WFRC/MAG/UDOT/UTA" 


dfRoutePrefix = pd.DataFrame([
    [   1,   14, 'Hwy'],
    [  15,   15, 'I-' ],
    [  16,   69, 'Hwy'],
    [  70,   70, 'I-' ],
    [  71,   79, 'Hwy'],
    [  80,   80, 'I-' ],
    [  81,   83, 'Hwy'],
    [  84,   84, 'I-' ],
    [  85,  214, 'Hwy'],
    [ 215,  215, 'I-' ],
    [ 216,  999, 'Hwy'],
    [1000, 5000, 'FA' ]
], columns=('routeFrom','routeTo','prefix'))
dfRoutePrefix

,routeFrom,routeTo,prefix
0,1,14,Hwy
1,15,15,I-
2,16,69,Hwy
3,70,70,I-
4,71,79,Hwy
5,80,80,I-
6,81,83,Hwy
7,84,84,I-
8,85,214,Hwy
9,215,215,I-


In [6]:
dfRoutes = sdfCombinedForecasts.groupby(['ROUTE','CO_FIPS'],as_index=False).agg(numSeg=('SEGID','count'))
dfRoutes['CO_FIPS'  ] = dfRoutes['CO_FIPS'].astype(int)
dfRoutes['ROUTE_numeric'] = pd.to_numeric(dfRoutes['ROUTE'], errors='coerce')

dfRoutes

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric
0,0006,49,36,6.0
1,0013,3,10,13.0
2,0015,3,5,15.0
3,0015,11,19,15.0
4,0015,35,27,15.0
...,...,...,...,...
736,UTA,57,3,NaN
737,WFRC,3,24,NaN
738,WFRC,11,47,NaN
739,WFRC,35,300,NaN


In [7]:
# Step 1: Add a dummy key to both DataFrames
dfRoutes['key'] = 1
dfRoutePrefix['key'] = 1

# Step 2: Merge on the dummy key
merged_df = pd.merge(dfRoutes, dfRoutePrefix, on='key')

# Step 3: Filter rows where ROUTE_numeric falls within the routeFrom and routeTo range
# Modify the filter to preserve NaN values (Step 3)
# We use | (OR) to add a condition that explicitly checks for NaNs in ROUTE_numeric
dfRoutePrefixWithPrefix = merged_df[((merged_df['ROUTE_numeric'] >= merged_df['routeFrom']) & 
                                     (merged_df['ROUTE_numeric'] <= merged_df['routeTo'])) |
                                     (merged_df['ROUTE_numeric'].isna())].copy()

# Fill specific columns with '' where ROUTE_numeric is NaN
# Adjust 'your_column_name_here' to the actual column(s) you want to fill for NaN ROUTE_numeric rows
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'prefix'   ] = dfRoutePrefixWithPrefix['ROUTE']
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeFrom'] = 0
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeTo'  ] = 0

dfRoutePrefixWithPrefix['ROUTE_numeric'].fillna(0, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop_duplicates()

# Step 4: Drop the dummy key column
dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop(columns=['key'])
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix
0,0006,49,36,6.0,1,14,Hwy
12,0013,3,10,13.0,1,14,Hwy
25,0015,3,5,15.0,15,15,I-
37,0015,11,19,15.0,15,15,I-
49,0015,35,27,15.0,15,15,I-
...,...,...,...,...,...,...,...
8832,UTA,57,3,0.0,0,0,UTA
8844,WFRC,3,24,0.0,0,0,WFRC
8856,WFRC,11,47,0.0,0,0,WFRC
8868,WFRC,35,300,0.0,0,0,WFRC


In [8]:
dfRoutePrefixWithPrefix['ROUTE_text'] = dfRoutePrefixWithPrefix['ROUTE_numeric'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)
dfRoutePrefixWithPrefix.loc[(dfRoutePrefixWithPrefix['ROUTE_text']=="0"), 'ROUTE_text'] = ''
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix,ROUTE_text
0,0006,49,36,6.0,1,14,Hwy,6
12,0013,3,10,13.0,1,14,Hwy,13
25,0015,3,5,15.0,15,15,I-,15
37,0015,11,19,15.0,15,15,I-,15
49,0015,35,27,15.0,15,15,I-,15
...,...,...,...,...,...,...,...,...
8832,UTA,57,3,0.0,0,0,UTA,
8844,WFRC,3,24,0.0,0,0,WFRC,
8856,WFRC,11,47,0.0,0,0,WFRC,
8868,WFRC,35,300,0.0,0,0,WFRC,


In [9]:
dfRoutePrefixWithPrefix.fillna('',inplace=True)
dfRoutePrefixWithPrefix['label']=dfRoutePrefixWithPrefix['prefix'] + dfRoutePrefixWithPrefix['ROUTE_text']

dfRoutePrefixWithPrefix.rename(columns={'ROUTE':'R','CO_FIPS':'C'}, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix[['R','C','label']]

dfRoutePrefixWithPrefix

,R,C,label
0,0006,49,Hwy6
12,0013,3,Hwy13
25,0015,3,I-15
37,0015,11,I-15
49,0015,35,I-15
...,...,...,...
8832,UTA,57,UTA
8844,WFRC,3,WFRC
8856,WFRC,11,WFRC
8868,WFRC,35,WFRC


In [10]:
# export to JSON
dfRoutePrefixWithPrefix.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/routes.json', orient='records', indent=4)

# Create Forecasts json

In [11]:
sdfCombinedForecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0006_141.0,0006,141.035,146.868,5.843301,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,1,0,0006_146.9,0006,146.868,149.902,3.039023,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,2,0,0006_149.9,0006,149.902,150.580,0.677983,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,3,0,0006_150.6,0006,150.580,152.555,1.973104,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,4,0,0006_152.6,0006,152.555,152.871,0.316275,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,4975,0,WFRC_8469,WFRC,0.000,0.000,0.567664,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{""paths"": [[[407213.3742000004, 4504030.9891],..."
4976,4976,0,WFRC_8470,WFRC,0.000,0.000,0.493148,35.0,1.0,WFRC,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{""paths"": [[[407723.2000000002, 4503422.753000..."
4977,4977,0,WFRC_8471,WFRC,0.000,0.000,0.405891,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{""paths"": [[[415734.59509999957, 4481608.6051]..."
4978,4978,0,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5,1.0245,0.0,"{""paths"": [[[424308.5, 4489341.199999999], [42..."


In [12]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^F\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 5]

# Display the list of columns
print(columns_matching_pattern)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justneededcols = sdfCombinedForecasts[['SEGID','CO_FIPS'] + columns_matching_pattern]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justneededcols

['F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050']


,SEGID,CO_FIPS,F2019,F2023,F2028,F2032,F2042,F2050
0,0006_141.0,49.0,1517.0,1600.0,1800.0,1800.0,2100.0,2400.0
1,0006_146.9,49.0,1517.0,1700.0,1800.0,1900.0,2100.0,2400.0
2,0006_149.9,49.0,2441.0,2600.0,2900.0,2900.0,3500.0,6100.0
3,0006_150.6,49.0,2441.0,2600.0,2900.0,3000.0,3600.0,6200.0
4,0006_152.6,49.0,2417.0,2500.0,2900.0,2900.0,3500.0,5900.0
...,...,...,...,...,...,...,...,...
4975,WFRC_8469,35.0,0.0,0.0,0.0,0.0,0.0,0.0
4976,WFRC_8470,35.0,0.0,0.0,0.0,0.0,0.0,0.0
4977,WFRC_8471,35.0,0.0,0.0,0.0,13500.0,16500.0,18500.0
4978,WFRC_8472,35.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justneededcols, id_vars=[col for col in sdfCombinedForecasts_justneededcols.columns if col not in columns_matching_pattern], 
                    value_vars=columns_matching_pattern, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[1:]).astype(int)

melted_df['F'] = melted_df['Value'].astype(int)

melted_df['CO_FIPS'] = melted_df['CO_FIPS'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justneededcols_melted = melted_df[['S','Y','F','CO_FIPS']]
sdfCombinedForecasts_justneededcols_melted

,S,Y,F,CO_FIPS
0,0006_141.0,2019,1517,49
1,0006_146.9,2019,1517,49
2,0006_149.9,2019,2441,49
3,0006_150.6,2019,2441,49
4,0006_152.6,2019,2417,49
...,...,...,...,...
29875,WFRC_8469,2050,0,35
29876,WFRC_8470,2050,0,35
29877,WFRC_8471,2050,18500,35
29878,WFRC_8472,2050,0,35


In [14]:
# export to JSON
sdfCombinedForecasts_justneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/forecasts.json', orient='records', indent=4)

# Create Observed

In [15]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^AADT\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern_AADT = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 8]

# Display the list of columns
print(columns_matching_pattern_AADT)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justAADTneededcols = sdfCombinedForecasts[['SEGID'] + columns_matching_pattern_AADT]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justAADTneededcols

['AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981']


,SEGID,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,...,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981
0,0006_141.0,1741.0,1760.0,1647.0,1517.0,1443.0,1458.0,1453.0,1385.0,1300.0,...,845.0,825.0,830.0,890.0,1060.0,1340.0,1240.0,965.0,1000.0,975.0
1,0006_146.9,1741.0,1760.0,1647.0,1517.0,1443.0,1458.0,1453.0,1385.0,1300.0,...,845.0,825.0,830.0,890.0,1060.0,1340.0,1240.0,965.0,1000.0,975.0
2,0006_149.9,2803.0,2834.0,2651.0,2441.0,2320.0,2610.0,2602.0,2480.0,2330.0,...,1230.0,1195.0,1205.0,1295.0,1435.0,1575.0,1505.0,1225.0,1225.0,1200.0
3,0006_150.6,2803.0,2834.0,2651.0,2441.0,2320.0,2610.0,2602.0,2480.0,2330.0,...,1230.0,1195.0,1205.0,1295.0,1435.0,1575.0,1505.0,1225.0,1225.0,1200.0
4,0006_152.6,2363.0,2339.0,2158.0,2417.0,2374.0,2350.0,2288.0,2175.0,2085.0,...,1730.0,1575.0,0.0,1910.0,2050.0,2190.0,2120.0,1840.0,1840.0,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,WFRC_8469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4976,WFRC_8470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4977,WFRC_8471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4978,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
sdfCombinedForecasts_justAADTneededcols.columns

Index(['SEGID', 'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018',
       'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012',
       'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006',
       'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000',
       'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994',
       'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988',
       'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982',
       'AADT1981'],
      dtype='object')

In [17]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justAADTneededcols, id_vars=[col for col in sdfCombinedForecasts_justAADTneededcols.columns if col not in columns_matching_pattern_AADT], 
                    value_vars=columns_matching_pattern_AADT, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[4:]).astype(int)

melted_df = melted_df[melted_df['Year']>=2000]

melted_df['O'] = melted_df['Value'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justAADTneededcols_melted = melted_df[['S','Y','O']]
sdfCombinedForecasts_justAADTneededcols_melted

,S,Y,O
0,0006_141.0,2022,1741
1,0006_146.9,2022,1741
2,0006_149.9,2022,2803
3,0006_150.6,2022,2803
4,0006_152.6,2022,2363
...,...,...,...
114535,WFRC_8469,2000,0
114536,WFRC_8470,2000,0
114537,WFRC_8471,2000,0
114538,WFRC_8472,2000,0


In [18]:
# export to JSON
sdfCombinedForecasts_justAADTneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/observed.json', orient='records', indent=0)

# create segments json

In [37]:
dfSegments = pd.merge(sdfCombinedForecasts[['SEGID','ROUTE','BMP']],dfRoutePrefixWithPrefix, left_on='ROUTE', right_on='R')
dfSegments.rename(columns={'SEGID':'S','label':'N'},inplace=True)

# set label
dfSegments['L'] = dfSegments['N'] + ' ~MP' + dfSegments['BMP'].round(1).astype(str)
dfSegments.loc[(dfSegments['R'].isin(['WFRC','MAG','CACHE','DIXIE','UDOT','UTA'])), 'L'] = dfSegments['S']


dfSegments.drop(columns=['ROUTE','BMP'],inplace=True)
dfSegments

,S,R,C,N,L
0,0006_141.0,0006,49,Hwy6,Hwy6 ~MP141.0
1,0006_146.9,0006,49,Hwy6,Hwy6 ~MP146.9
2,0006_149.9,0006,49,Hwy6,Hwy6 ~MP149.9
3,0006_150.6,0006,49,Hwy6,Hwy6 ~MP150.6
4,0006_152.6,0006,49,Hwy6,Hwy6 ~MP152.6
...,...,...,...,...,...
8524,WFRC_8472,WFRC,57,WFRC,WFRC_8472
8525,WFRC_8473,WFRC,3,WFRC,WFRC_8473
8526,WFRC_8473,WFRC,11,WFRC,WFRC_8473
8527,WFRC_8473,WFRC,35,WFRC,WFRC_8473


In [39]:
# export to JSON
dfSegments.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/segments.json', orient='records', indent=0)